In [1]:
import os, shutil
parent_dir = "./dataset/"
folders = os.listdir(parent_dir + "/multiclass") # Path to Training Images
print(folders)

['cats', 'dogs', 'horses', 'humans']


## Seperating Training data from Testing data

In [2]:
classes = ["dogs", "cats", "horses", "humans"] # Labels

# First create a validation folder
if not os.path.isdir(parent_dir + "/val_images"):
    os.mkdir(parent_dir + "/val_images")

# Now create a seperate folder for each label
for label in classes:
    os.mkdir(parent_dir + "/val_images/" + label)


In [3]:
for each_class in classes: # Iterate over each class
    image_path = os.path.join(parent_dir + "val_images", each_class)# creating a seperate folder for each class in validation folder
 
  if not image_path:
    os.mkdir(image_path)

SPLIT = 0.9 # spliting data into 90:10 

# Load all images from training data and make a split
for image_folder in os.listdir(parent_dir + "/multiclass"):
    Path = parent_dir + "/multiclass/" + image_folder
    images = os.listdir(Path)# Load all the images
    split_size =  int(SPLIT * len(images))
    images_to_move = images[split_size :] # Train-Test split
  #print(len(images_to_move))
    for validation_image in images_to_move:
        src = os.path.join(Path, validation_image) 
        dest = os.path.join(parent_dir + "/val_images/" + image_folder, validation_image)
        shutil.move(src, dest) # Moving selected images

### ```Now how many images left in training dataset```

In [28]:
for folder in folders:
    Path = parent_dir + "/multiclass/" + folder
    print(folder + " " + str(len(os.listdir(Path))))

cats 181
dogs 181
horses 181
humans 181


### ```And what about validation dataset```

In [29]:
validation_folder = os.listdir(parent_dir + "/val_images/")
for folder in validation_folder:
    Path = parent_dir + "/val_images/" + folder
    print(folder + " " + str(len(os.listdir(Path))))

cats 21
dogs 21
horses 21
humans 21


### ``` Train data generator```

In [30]:
from keras.preprocessing.image import ImageDataGenerator

### ```Training Data Generator```

In [31]:
train_gen = ImageDataGenerator(rescale = 1.0/255)

train_generator = train_gen.flow_from_directory(parent_dir + "/multiclass/",
                                                target_size = (300, 300),
                                                batch_size = 32,
                                                class_mode = "categorical"
                                                )

Found 724 images belonging to 4 classes.


### ``` Testing Data Generator```

In [32]:
valid_generator = train_gen.flow_from_directory(parent_dir + "/val_images/",
                                               target_size = (300, 300),
                                               batch_size = 32,
                                               class_mode = "categorical"
                                               )

Found 84 images belonging to 4 classes.


### ```Lets have a look how train_gen works```

In [34]:
for x, y in train_generator:
    print(x.shape)
    print(y.shape)
    break

(32, 300, 300, 3)
(32, 4)


### ``` Building a CNN Model```

In [35]:
from keras.layers import *
from keras.models import Sequential

In [36]:
from keras import optimizers 

adam = optimizers.Adam(learning_rate = 1e-4)

In [61]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation = "relu", input_shape = (300, 300, 3)))
model.add(MaxPool2D(2, 2))
model.add(Conv2D(64, (3, 3), activation = "relu"))
model.add(MaxPool2D(2, 2))
model.add(Conv2D(128, (3, 3), activation = "relu"))
model.add(MaxPool2D(2, 2))
model.add(Conv2D(128, (3, 3), activation = "relu"))
model.add(MaxPool2D(2, 2))
model.add(Flatten())
model.add(Dense(64, activation = "relu"))
model.add(Dense(4, activation = "softmax"))
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 298, 298, 32)      896       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 147, 147, 64)      18496     
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 71, 71, 128)       73856     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 33, 33, 128)      

In [62]:
model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics = ["accuracy"])

In [63]:
import warnings
warnings.filterwarnings("ignore")

In [64]:
history = model.fit_generator(train_generator,
                             epochs = 80,
                            
                             validation_data = valid_generator,
                    )

Epoch 1/80
 2/23 [=>............................] - ETA: 2s - loss: 1.3945 - accuracy: 0.2188WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0568s vs `on_train_batch_end` time: 0.1077s). Check your callbacks.


23/23 [==============================] - 4s 181ms/step - loss: 1.3507 - accuracy: 0.3177 - val_loss: 1.2978 - val_accuracy: 0.2976
Epoch 2/80
23/23 [==============================] - 4s 171ms/step - loss: 1.1414 - accuracy: 0.5180 - val_loss: 1.1496 - val_accuracy: 0.5476
Epoch 3/80
23/23 [==============================] - 4s 172ms/step - loss: 0.9171 - accuracy: 0.6298 - val_loss: 1.2327 - val_accuracy: 0.4762
Epoch 4/80
23/23 [==============================] - 4s 172ms/step - loss: 0.8032 - accuracy: 0.6837 - val_loss: 1.1470 - val_accuracy: 0.5238
Epoch 5/80
23/23 [==============================] - 4s 171ms/step - loss: 0.6736 - accuracy: 0.7251 - val_loss: 1.1462 - val_accuracy: 0.5357
Epoch 6/80
23/23 [==============================] - 4s 172ms/step - loss: 0.5512 - accuracy: 0.8052 - val_loss: 1.0847 - val_accuracy: 0.5595
Epoch 7/80
23/23 [==============================] - 4s 173ms/step - loss: 0.4756 - accuracy: 0.8177 - val_loss: 1.1168 - val_accuracy: 0.5714
Epoch 8/80
23/23 

23/23 [==============================] - 5s 207ms/step - loss: 5.2570e-04 - accuracy: 1.0000 - val_loss: 3.3917 - val_accuracy: 0.5595
Epoch 59/80
23/23 [==============================] - 5s 209ms/step - loss: 4.9819e-04 - accuracy: 1.0000 - val_loss: 3.3989 - val_accuracy: 0.5357
Epoch 60/80
23/23 [==============================] - 6s 256ms/step - loss: 4.7244e-04 - accuracy: 1.0000 - val_loss: 3.4003 - val_accuracy: 0.5357
Epoch 61/80
23/23 [==============================] - 6s 249ms/step - loss: 4.5318e-04 - accuracy: 1.0000 - val_loss: 3.4531 - val_accuracy: 0.5476
Epoch 62/80
23/23 [==============================] - 5s 239ms/step - loss: 4.3688e-04 - accuracy: 1.0000 - val_loss: 3.4505 - val_accuracy: 0.5357
Epoch 63/80
23/23 [==============================] - 5s 208ms/step - loss: 4.1638e-04 - accuracy: 1.0000 - val_loss: 3.4390 - val_accuracy: 0.5357
Epoch 64/80
23/23 [==============================] - 5s 199ms/step - loss: 3.9666e-04 - accuracy: 1.0000 - val_loss: 3.4784 - val_